<a href="https://colab.research.google.com/github/Maplemx/Agently/blob/main/playground/constrast_between_Agently_workflow_and_LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Constrast between Agently Workflow and LangGraph

**Author:** Agently Team

## Intro 简介

This document is a not-complete-compared introduction between Agently Workflow and LangGraph. We wrote this document due to many community developers wondering the differences between two framework modules. But because of the lack of time, we may have some lack of information or mistakes. If you discover those, please [report issues here](https://github.com/Maplemx/Agently/issues/new) or contact us, we would like to change this document after comfirming the mistakes to make this document objective.

本文档是Agently Workflow和LangGraph的不全面比较文档。因为很多社区开发者想要了解这两个框架模块之间的差异，我们撰写了这篇文档。但由于准备时间以及对LangGraph框架了解的深度限制，我们也可能在对比时有所纰漏和错误。如果您发现了这些问题，可以[点击这里向我们提交](https://github.com/Maplemx/Agently/issues/new)，或是用其他方式与我们去得联系。我们在确认问题后，会及时修改文档内容，来确保文档的客观性。

## Compare Table 功能对比表

|Feature|Agently Workflow|LangGraph|
|---|---|---|
|**Create Workflow Instance**<br /><br />**创建工作流对象**|✅<br />Created by `Agently.Workflow()`<br /><br />由`Agently.Workflow()`类创建|✅<br />Created by `langgraph.graph.StateGraph()`<br />Customized structure class `State` is required for workflow data communication.<br /><br />通过`langgraph.graph.StateGraph()`类创建<br />需要传入自定义结构类`State`作为补充，用于后续工作流中的数据传递结构|
|**Define Workflow Chunk/Node**<br /><br />**定义工作流块/节点**|✅ 🚀<br />Defined by function decorator `@workflow.chunk()`<br />Defined chunks will be found in dict `workflow.chunks`<br />Default named by function name.<br /><br />由装饰器`@workflow.chunk()`装饰执行函数完成定义<br />定义完成的工作块将可以在字典`workflow.chunks`中找到<br />工作块默认命名与执行函数命名一致|✅<br />Define node function directly<br />Register as a node by `workflow.add_node("node name", node_func)`<br /><br />直接定义函数<br />通过`workflow.add_node("node name", node_func)`方法注册成为一个node节点|
|**Executor Function Compatibility**<br />**工作流块/节点运行函数兼容性**|🟠<br />Any Modules<br />Support async `workflow.start_async()`<br />Support streaming handler using listener<br />Not support streaming output using generator<br /><br />支持在运行函数内使用任何其他模块<br />支持异步`workflow.start_async()`<br />支持使用监听器处理流式输出<br />不支持使用generator进行流式输出|✅<br />Any Modules<br />Support async `app.ainvoke()`<br />Support streaming output using generator `app.astream()`<br /><br />支持在运行函数内使用任何其他模块<br />支持异步 `app.ainvoke()`<br />支持使用generator进行流式输出`workflow.astream()`|
|**Start and End Points**<br /><br />**开始和结束**|✅ 🚀<br />Provide standard chunk type `Start` and `END`<br />Those two standard chunks will be put into `workflow.chunks` by default when workflow is created<br /><br />提供标准的chunk_type，`Start`和`End`<br />两个名为"start"和"end"的标准工作块在工作流创建时会默认内置到工作块池中|✅<br />Using `workflow.set_entry_ponit("node name", node_func)` to set the node to start<br />Using `langgraph.graph.END` as the ending node<br /><br />通过`workflow.set_entry_ponit("node name", node_func)`的方式确定起点块<br />使用内置的langgraph.graph.END作为终点块|
|**Normal Connection**<br /><br />**普通连接**|✅ 🚀<br />Connect two chunks using`workflow.chunks["from_chunk_name"].connect_to(workflow.chunks["to_chunk_name"])`<br />Simplify expression: `.connect_to("to_chunk_name")`<br />Chain expression like `.connect_to("chunk_a").connect_to("chunk_b")` supported.<br /><br />使用`workflow.chunks["from_chunk_name"].connect_to(workflow.chunks["to_chunk_name"])`连接<br />支持使用`.connect_to("to_chunk_name")`方式简写<br />支持`.connect_to("chunk_a").connect_to("chunk_b")`这样的链式表达|✅<br />Connect two nodes using `workflow.add_edge("from_node_name", "to_node_name")`<br /><br />使用`workflow.add_edge("from_node_name", "to_node_name")`连接两个node节点|
|**Conditional Connection**<br /><br />**条件连接**|🟠<br />Using expression like `.if_condition(lambda value, storage: value==1)` in chain expression<br />Get return value from last chunk as `value` and get workflow storage as `storage`<br />Support `.if_condition()`, `.else_condition()` currently<br />Can not support more than 2 conditions branches right now or use more "if-else" combination to express but we're working on it<br /><br />使用类似`.if_condition(lambda value, storage: value==1)`的形式进行条件判断表达<br />从上一个连接块获取函数return的返回值作为`value`，使用工作流全局存储作为`storage`<br />目前只能支持不超过两种情况的分支，或是使用多个if-else组合来进行表达，我们正在改进这一点|✅ 🚀<br />Using `workflow.add_conditional_edges("from_node_name", condition_func, { "return_value_1": "to_node_name_1", ... })` to define conditional edges<br />Support return more than 2 conditions to branch<br /><br />使用`workflow.add_conditional_edges("from_node_name", condition_func, { "return_value_1": "to_node_name_1", ... })`进行表达<br />支持在一个条件判断函数返回2个以上的条件结果，并进行分支规划|
|**Ring Connection**<br />**成环支持**|✅<br />Yes, you can connect chunks into a ring to repeatly processing<br />You can use `if_condition()` to break the ring<br /><br />支持，你可以将多个工作块首尾相连形成环状来多次执行<br />你可以使用`if_condition()`进行条件判断来跳出环|✅<br />Yes, you can connect nodes into a ring to repeatly processing<br />You can use `workflow.add_conditional_edges()` to break the ring<br /><br />支持，你可以将多个工作节点首尾相连形成环状来多次执行<br />你可以使用`workflow.add_conditional_edges()`进行条件判断来跳出环|
|**Parallel Branch**<br />**并行分支**|✅<br />You can connect one chunk to two or more different chunks and connect all these chunks to different handles of the end chunk to make a parallel branch<br /><br />你可以通过将一个工作块连接多个下游块，并将这些下游块连接到终点块的不同端点上（不同端点才会等待上游全部任务的完成）来构造并行分支|❌<br />Can not support, if you try to connect one node to two or more different nodes you will receive an error<br /><br />不支持，如果你尝试将一个node下游同时连接多个node，你会收到一个错误警告|
|**Loop by List**<br />**基于列表的循环**|✅<br />Using `.loop_with(sub_workflow)` to pass items in list to sub workflow one by one and get a list return result<br /><br />支持使用`.loop_with(sub_workflow)`将上游块的列表中的元素一个一个地传递给子工作流sub workflow，并从子工作流获得汇集成列表的返回结果|❌<br />Not found in quick start guide book.<br /><br />在快速入门的相关文档中未找到|
|**Start Workflow and Initial Data**<br />**启动工作流及启动时数据传递**|✅<br />Using `result = workflow.start(initial_inputs, storage=initial_storage)` to start workflow and get final result from chunk "end"'s return<br />Passing `initial_inputs` to parameter `inputs` of next chunk's executor function<br />Setting `initial_storage` to workflow's global storage<br /><br />使用`result = workflow.start(initial_inputs, storage=initial_storage)`的方式启动工作流<br />将`initial_inputs`通过工作块执行函数的`inputs`参数传给第一个处理工作块<br />将`initial_storage`作为工作流全局数据`storage`的初始数据|✅<br />Using `app = workflow.compile()` to compile workflow first<br />Using `final_state = app.invoke({"state_key": state_value}, config={})` to start workflow and get final state as result after workflow processing<br />Passing initial state data in `app.invoke()` to set initial state<br /><br />需要使用`app = workflow.compile()`将工作流事先进行编译<br />通过`final_state = app.invoke({"state_key": state_value}, config={})`启动工作流，并将运行后的最终state数据返回<br />在`app.invoke()`方法中传递初始的state数据|
|**Runtime Data Communication**<br />**运行时数据通讯**|✅<br />Using `inputs` to passing data from last chunk to next chunk<br />Using different handles of chunk to manage data passing and running orders<br />Using `stroage` to store workflow global data<br /><br />使用`inputs`在具有直接连接顺序关系的工作块之间进行数据传递<br />（提供强时序相关的数据的可靠传递方式）<br />在一个工作块上使用多个端点（handles）进行数据传递分组及执行顺序控制<br />（如具有两个输入端点的工作块会等待两个端点对应的前置分支流完成处理后才开始工作）<br />使用`storage`进行工作流全局数据传递|🟠<br />Using pre-stated `state` only and you can not customize `state`'s structure in workflow<br />只使用预先定义好结构的`state`进行工作过程数据传递，并且工作流处理过程中不能修改`state`的结构。<br />（无法应对复杂处理过程或是多分支时序依赖较高的场景，因为这时`state`键值的状态不确定性变高）<br />（这可能也是LangGraph没有支持并行分支的原因）|
|**Saving Data and Restore**<br /><br />**工作数据保存和恢复**|✅<br />Using `workflow.storage.get_all()` to get storage data and using `workflow.storage.set_with_dict()` to reset storage data<br /><br />支持使用`workflow.storage.get_all()`取出工作流的全局数据，并用`workflow.storage.set_with_dict()`更新工作流全局数据|✅ 🚀<br />Using `checkpointer = langgraph.checkpoint.MemorySaver()` to persist state between graph runs<br /><br />使用`checkpointer = langgraph.checkpoint.MemorySaver()`在多次工作流运行间保存状态数据|
|**Workflow Graph**<br />**工作流图形生成**|✅<br />Using `workflow.draw()` to get mermaid code<br /><br />支持通过`workflow.draw()`获取工作流的Mermaid代码|✅ 🚀<br />Compile workflow by `app = workflow.compile()` first then use `app.get_graph().draw_mermaid()` to get mermaid code<br />Support ASCII, Mermaid, PNG<br /><br />通过`app = workflow.compile()`对工作流进行编译后，可以通过`app.get_graph().draw_mermaid()`的方式生成图形代码<br />除了Mermaid外，还支持ASCII和PNG格式输出|
|**Framework Ecosystem**<br />**框架生态**|✅<br />Agently Settings to switch between models speedily without changing any logic code<br />Agently Agent Request to make model request easily with solid format control<br />Agently P-YAML to seperate coding logic and model requesting prompt<br />English and Chinese language support<br /><br />Agently Settings帮助开发者不修改任何业务逻辑代码，使用简单配置就能切换驱动模型<br />Agently Agent Request帮助开发者直观轻松地对模型请求进行表达，并获得高稳定性的结构化数据输出<br />Agently P-YAML帮助开发者将模型请求提示优化等工作从业务代码中分离出来<br />提供英文及中文双语支持|✅<br />World famous LangChain framework<br />LangSmith and LangServer to help developers monitor and deploy application<br />English support only<br /><br />LangChain是世界知名框架<br />LangSmith和LangServer能帮助开发者进行应用运行过程监控和服务部署<br />只提供英文支持|

图例说明：
✅ 完全支持此能力
❌ 不支持此能力或较为复杂
🟠 部分支持此能力，或在支持此能力时有瑕疵，需要改进
🚀 在支持此能力时，提供了便利易用的方法

## Code Compatation 实现代码对比

### Subject Settings 命题设计

We'll try to cover as many abilities from the table above as possible to present differences between two frameworks.

为了更好展现不同框架的特点，我们设计的命题将尽可能多地覆盖上表中的相关能力。

<img width="1024" src="https://github.com/Maplemx/Agently/assets/4413155/392bb081-99c5-44ba-ab64-c752dc8443ae" />

**Task Description**

1. Pass data `times=0` into workflow when workflow start
2. Ask user to input some string
3. Start two parallel branches:

    - Branch A:
        1. Add `times` by 1
    
    - Branch B:
        1. Split user input string character by character into a list
        2. Loop the character item in the list and print character item one by one (In real task this sub workflow should be much more complex)
        3. Collect character items from sub workflow and regroup them into a string, print it and append it into list `all_user_inputs`.

4. When two branches all finish their works, make a judgement about value of `times`, if `times>=3` then end this workflow, else go back to step 2.
5. Get `all_user_inputs` after workflow finish and print it.

**任务描述**

1. 将数据`times=0`在工作流开始工作时，传递到工作流内部
2. 请用户输入一个若干字符长度的字符串
3. 开启两个平行分支：

    - 分支A：
        1. 将`times`的值增加1

    - 分支B：
        1. 将用户输入的字符串拆分成由单个字符组成的list
        2. 循环访问list中的每一个元素，进行相同的处理过程：打印当前元素信息（在真实的任务中，这个子工作流的任务复杂度其实会高很多）
        3. 回收子工作流中的字符元素并将它们重组为一个字符串，然后把字符串打印出来，并把字符串放入list`all_user_inputs`中存储

4. 当分支A和B都完成后，做一个关于`times`当前值的判断，如果`times>=3`那么工作流结束，否则回到步骤2
5. 在工作流执行完毕后，获取`all_user_inputs`的结果并打印出来

### Install Packages

In [ ]:
!pip install -q -U langgraph Agently mermaid-python

### LangGraph

In [4]:
from langgraph.graph import StateGraph, END

# Define State Class
from typing import TypedDict, Optional

## I have to go back here to modified this class
## when I write node functions down below time to time
## because I find out that I need to pass more other data in process
class State(TypedDict):
    times: Optional[int] = None
    user_input_data: Optional[str] = None
    splited_list: Optional[list] = None
    all_user_inputs: Optional[list] = None

# Create Workflow Instance
workflow = StateGraph(State)

# Define Node Functions
def start(state):
    return

def user_input(state):
    return { "user_input_data": input("[Input String]: ") }

def add_times(state):
    return { "times": state.get("times") + 1 }

def split_input_to_list(state):
    user_input_data = state.get("user_input_data")
    splited_list = []
    for char in user_input_data:
        splited_list.append(char)
    return { "splited_list": splited_list }

## I can't use a loop using LangGraph here so I just combine
## `loop`, `print_char` item by item, `regroup_input` 3 steps
## into this one
def combined_steps(state):
    splited_list = state.get("splited_list")
    regrouped_input = ""
    for char in splited_list:
        print(char)
        regrouped_input += char
    print(regrouped_input)
    all_user_inputs = state.get("all_user_inputs") or []
    all_user_inputs.append(regrouped_input)
    return { "all_user_inputs": all_user_inputs }

# Register Node
workflow.add_node("start", start)
workflow.add_node("user_input", user_input)
workflow.add_node("split_input_to_list", split_input_to_list)
workflow.add_node("combined_steps", combined_steps)
workflow.add_node("add_times", add_times)

# Connection
workflow.set_entry_point("start")
workflow.add_edge("start", "user_input")
workflow.add_edge("user_input", "split_input_to_list")
## the connections seem simple because I warpped
## the loop sub workflow logic into `combined_steps`
workflow.add_edge("split_input_to_list", "combined_steps")
## I can not build parallel branches so I just make them run in orders
## In this case that doesn't matter but maybe not in other cases
workflow.add_edge("combined_steps", "add_times")
workflow.add_conditional_edges(
  "add_times",
  lambda state: "end" if state.get("times") >= 3 else "continue",
  {
    "end": END,
    "continue": "user_input"
  }
)

# Compile
app = workflow.compile()

In [2]:
# Draw Mermaid Workflow Graph
from mermaid import Mermaid
Mermaid(app.get_graph().draw_mermaid())

In [5]:
# Start Workflow and Get Result
result = app.invoke({ "times": 0 })
print(result["all_user_inputs"])

[Input String]: LangGraph
L
a
n
g
G
r
a
p
h
LangGraph
[Input String]: and
a
n
d
and
[Input String]: Agently
A
g
e
n
t
l
y
Agently
['LangGraph', 'and', 'Agently']


### Agently Workflow

In [2]:
import Agently

# Create Workflow Instance
workflow = Agently.Workflow()
sub_workflow = Agently.Workflow()

# Define Chunks
## (Including define chunk function and register chunk)
@workflow.chunk()
def user_input(inputs, storage):
    return input("[Input String]: ")

@workflow.chunk()
def add_times(inputs, storage):
    storage.set("times", storage.get("times") + 1)
    return

## Need a wait point chunk to wait two branches to finish
@workflow.chunk()
def wait_point(inputs, storage):
    return

@workflow.chunk()
def split_input_to_list(inputs, storage):
    user_input_data = inputs["default"]
    splited_list = []
    for char in user_input_data:
        splited_list.append(char)
    return splited_list

## Use sub_workflow to handle items from `split_input_to_list`
@sub_workflow.chunk()
def print_char(inputs, storage):
    print(inputs["default"])
    return inputs["default"]

@workflow.chunk()
def regroup_input(inputs, storage):
    regrouped_input = ""
    for item in inputs["default"]:
        regrouped_input += item["default"]
    print(regrouped_input)
    return regrouped_input

@workflow.chunk()
def append_to_all_user_inputs(inputs, storage):
    all_user_inputs = storage.get("all_user_inputs", [])
    all_user_inputs.append(inputs["default"])
    storage.set("all_user_inputs", all_user_inputs)
    return

# Connection
## Sub Workflow in Loop
sub_workflow.connect_to("print_char").connect_to("end")

## Branch A
(
    workflow
        .connect_to("user_input")
        .connect_to("add_times")
        ## Use different handle to wait different branch
        .connect_to("wait_point.branch_a")
        .if_condition(lambda value, storage: storage.get("times") >= 3)
            .connect_to("end")
        .else_condition()
            .connect_to("user_input")
)

## Branch B
(
    ## Only define connections those were not defined yet
    ## So, start from `user_input`
    workflow.chunks["user_input"]
        .connect_to("split_input_to_list")
        ## Use .loop_with() to call sub workflow
        .loop_with(sub_workflow)
        .connect_to("regroup_input")
        .connect_to("append_to_all_user_inputs")
        ## Use different handle to wait different branch
        .connect_to("wait_point.branch_b")
)

In [21]:
# Draw Mermaid Workflow Graph
## Yep, I can't use Mermaid() to draw it because it's complex
print(workflow.draw())

%%{ init: { 'flowchart': { 'curve': 'linear' }, 'theme': 'neutral' } }%%
%% Rendered By Agently %%
flowchart LR
classDef chunk_style fill:#fbfcdb,stroke:#666,stroke-width:1px,color:#333;
classDef loop_style fill:#f5f7fa,stroke:#666,stroke-width:1px,color:#333,stroke-dasharray: 5 5
    subgraph Loop_1
    direction LR
    eeefd9bb-48f9-4c67-a65d-db73a5fe5a1b("start"):::chunk_style -.-> |"* -->-- default"| e1994333-d244-4853-b8ef-cce16183c074("print_char"):::chunk_style
    e1994333-d244-4853-b8ef-cce16183c074("print_char"):::chunk_style -.-> |"* -->-- default"| 1298e0ae-080d-43b8-9c17-cb08e6952274("end"):::chunk_style
    end
    886d5aff-c668-4b51-b5e0-a7985b3850f8("start"):::chunk_style -.-> |"* -->-- default"| 72a42165-539d-438c-b07a-02c6dda7be64("user_input"):::chunk_style
    72a42165-539d-438c-b07a-02c6dda7be64("user_input"):::chunk_style -.-> |"* -->-- default"| 7a03950a-5b10-4667-b5d5-e8af6740bfc6("add_times"):::chunk_style
    7a03950a-5b10-4667-b5d5-e8af6740bfc6("add_times")::

<image width="1024" src="https://github.com/Maplemx/Agently/assets/4413155/91ddf134-31d5-4afe-9142-6e94e59d022f" />

In [3]:
# Start Workflow and Get Result
workflow.start(storage = { "times": 0 })
print(workflow.storage.get("all_user_inputs"))

[Input String]: LangGraph
L
a
n
g
G
r
a
p
h
LangGraph
[Input String]: and
a
n
d
and
[Input String]: Agently
A
g
e
n
t
l
y
Agently
['LangGraph', 'and', 'Agently']


---

[**_<font color = "red">Agent</font><font color = "blue">ly</font>_** Framework - Speed up your AI application development](https://github.com/Maplemx/Agently)